In [33]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [34]:
# Load the dataset
df = pd.read_csv("heart.csv")

In [35]:
# Select the relevant features for K-means (excluding "Time" and "Class")
X = df.iloc[:, 0:13]  # Select columns V1 to V28 and Amount
y = df.iloc[:,13]

In [36]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
# Standardize the feature values (optional but often helpful)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [38]:
# Define the VAE architecture
original_dim = X_train.shape[1]
latent_dim = 2

In [39]:
# Encoder
encoder_inputs = keras.layers.Input(shape=(original_dim,))
encoder_layer = keras.layers.Dense(128, activation='relu')(encoder_inputs)
z_mean = keras.layers.Dense(latent_dim)(encoder_layer)
z_log_var = keras.layers.Dense(latent_dim)(encoder_layer)

In [40]:
# Sampling layer
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.shape(z_mean)[0], latent_dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = keras.layers.Lambda(sampling)([z_mean, z_log_var])

In [41]:
# Decoder
decoder_inputs = keras.layers.Input(shape=(latent_dim,))
decoder_layer = keras.layers.Dense(128, activation='relu')(decoder_inputs)
outputs = keras.layers.Dense(original_dim, activation='sigmoid')(decoder_layer)

In [42]:
# Build the VAE model
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')
decoder = keras.Model(decoder_inputs, outputs, name='decoder')
vae_outputs = decoder(encoder(encoder_inputs)[2])
vae = keras.Model(encoder_inputs, vae_outputs, name='vae')

In [43]:
# Define the VAE loss
reconstruction_loss = tf.keras.losses.mean_squared_error(encoder_inputs, vae_outputs)
kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
vae_loss = reconstruction_loss + kl_loss

vae.add_loss(vae_loss)

In [50]:
vae.compile(optimizer='adam')

# Train the VAE model
vae.fit(X_train, epochs=10, batch_size=32, validation_split=0.20)

Epoch 1/10
21/21 [==============================] - 1s 10ms/step - loss: 0.9977 - val_loss: 1.0112
Epoch 2/10
21/21 [==============================] - 0s 4ms/step - loss: 0.9977 - val_loss: 1.0110
Epoch 3/10
21/21 [==============================] - 0s 3ms/step - loss: 0.9976 - val_loss: 1.0115
Epoch 4/10
21/21 [==============================] - 0s 3ms/step - loss: 0.9977 - val_loss: 1.0108
Epoch 5/10
21/21 [==============================] - 0s 3ms/step - loss: 0.9979 - val_loss: 1.0114
Epoch 6/10
21/21 [==============================] - 0s 4ms/step - loss: 0.9978 - val_loss: 1.0110
Epoch 7/10
21/21 [==============================] - 0s 4ms/step - loss: 0.9977 - val_loss: 1.0115
Epoch 8/10
21/21 [==============================] - 0s 4ms/step - loss: 0.9977 - val_loss: 1.0112
Epoch 9/10
21/21 [==============================] - 0s 3ms/step - loss: 0.9977 - val_loss: 1.0114
Epoch 10/10
21/21 [==============================] - 0s 3ms/step - loss: 0.9976 - val_loss: 1.0114


In [51]:
# Encode the data
encoded_X_train, _, _ = encoder.predict(X_train)
encoded_X_test, _, _ = encoder.predict(X_test)

# Build a disease prediction model using the encoded features
disease_model = keras.Sequential([
    keras.layers.Input(shape=(latent_dim,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')  # Use 'sigmoid' for binary classification
])

disease_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the disease prediction model
disease_model.fit(encoded_X_train, y_train, epochs=10, batch_size=32, validation_split=0.20)

# Evaluate the model
test_loss, test_accuracy = disease_model.evaluate(encoded_X_test, y_test)
print("Test accuracy:", test_accuracy)


7/7 [==============================] - 0s 2ms/step
Epoch 1/10
21/21 [==============================] - 1s 11ms/step - loss: 0.6928 - accuracy: 0.4985 - val_loss: 0.6917 - val_accuracy: 0.5244
Epoch 2/10
21/21 [==============================] - 0s 3ms/step - loss: 0.6920 - accuracy: 0.5137 - val_loss: 0.6917 - val_accuracy: 0.5244
Epoch 3/10
21/21 [==============================] - 0s 3ms/step - loss: 0.6918 - accuracy: 0.5137 - val_loss: 0.6914 - val_accuracy: 0.5244
Epoch 4/10
21/21 [==============================] - 0s 3ms/step - loss: 0.6916 - accuracy: 0.5274 - val_loss: 0.6914 - val_accuracy: 0.5305
Epoch 5/10
21/21 [==============================] - 0s 3ms/step - loss: 0.6914 - accuracy: 0.5168 - val_loss: 0.6910 - val_accuracy: 0.5244
Epoch 6/10
21/21 [==============================] - 0s 3ms/step - loss: 0.6910 - accuracy: 0.5137 - val_loss: 0.6910 - val_accuracy: 0.5244
Epoch 7/10
21/21 [==============================] - 0s 3ms/step - loss: 0.6909 - accuracy: 0.5137 - val_loss

In [55]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers

# Load the dataset
data = pd.read_csv("heart.csv")

# Split the data into features and target
X = data.drop("target", axis=1)
y = data["target"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the autoencoder model
input_dim = X_train.shape[1]
encoding_dim = 10  # Adjust the latent space dimension as needed

autoencoder = keras.Sequential([
    layers.Input(shape=(input_dim,)),
    layers.Dense(encoding_dim, activation='relu'),
    layers.Dense(input_dim, activation='sigmoid')
])

# Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
epochs = 20  # Adjust as needed
batch_size = 32

autoencoder.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, shuffle=True, validation_data=(X_test, X_test))

# Evaluate the autoencoder
test_loss = autoencoder.evaluate(X_test, X_test)
print(f"Test loss: {test_loss}")

# Use the trained autoencoder for reconstruction
reconstructed_data = autoencoder.predict(X_test)

# You can calculate accuracy or other metrics for the reconstruction
# Compare X_test and reconstructed_data to assess the autoencoder's performance


Epoch 1/20
26/26 [==============================] - 1s 7ms/step - loss: 1.2805 - val_loss: 1.3272
Epoch 2/20
26/26 [==============================] - 0s 3ms/step - loss: 1.2404 - val_loss: 1.2862
Epoch 3/20
26/26 [==============================] - 0s 3ms/step - loss: 1.2025 - val_loss: 1.2475
Epoch 4/20
26/26 [==============================] - 0s 3ms/step - loss: 1.1664 - val_loss: 1.2096
Epoch 5/20
26/26 [==============================] - 0s 4ms/step - loss: 1.1307 - val_loss: 1.1727
Epoch 6/20
26/26 [==============================] - 0s 5ms/step - loss: 1.0950 - val_loss: 1.1358
Epoch 7/20
26/26 [==============================] - 0s 4ms/step - loss: 1.0592 - val_loss: 1.0997
Epoch 8/20
26/26 [==============================] - 0s 3ms/step - loss: 1.0242 - val_loss: 1.0651
Epoch 9/20
26/26 [==============================] - 0s 3ms/step - loss: 0.9901 - val_loss: 1.0321
Epoch 10/20
26/26 [==============================] - 0s 2ms/step - loss: 0.9583 - val_loss: 1.0018
Epoch 11/20
26/26 [